# Pix2Pix 训练实验 - Google Colab 设置

本 notebook 用于在 Google Colab 上运行 Pix2Pix (L1 + GAN) 训练实验（E2）。

## 使用说明

1. **上传代码到 GitHub/Gitee**（推荐）或直接上传 zip 到 Colab
2. **在 Google Drive 上准备数据**：
   - 将 `processed` 和 `splits` 文件夹压缩成 `data.zip`
   - 上传 `data.zip` 到 Google Drive（建议放在 Drive 根目录）
3. **修改下面的 `DRIVE_DATA_ZIP`**：改为你的 `data.zip` 在 Drive 中的路径
4. **点击 "Run all"** 运行所有 cell

## 数据准备说明

- **压缩方式**：将 `data/processed` 和 `data/splits` 两个文件夹压缩成 `data.zip`
- **上传位置**：建议上传到 Drive 根目录 `/content/drive/MyDrive/data.zip`
- **解压后结构**：`data/processed/train/{photo,label}`, `data/processed/val/{photo,label}`, `data/splits/cityscapes_split_seed42.json`


## 1. 安装依赖


In [ ]:
# 安装必要的 Python 包
!pip install -q tqdm pillow torchvision scikit-image torchmetrics

# 如果需要 Perceptual Loss（可选）
# !pip install -q lpips

print("✅ 依赖安装完成")


## 2. 挂载 Google Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

print("✅ Google Drive 挂载完成")


## 3. 克隆代码仓库（或上传代码）


In [ ]:
# 方式1：从 GitHub/Gitee 克隆（推荐）
# 请将下面的 URL 替换为你的仓库地址
# !git clone https://github.com/zhuxie0719/Image-to-Image-Translation-Experiment
# %cd Image-to-Image-Translation-Experiment

# 方式2：如果代码已经在 Colab 中（通过上传 zip 等方式）
# 直接使用当前目录，跳过这一步

import os
import shutil
import subprocess
from pathlib import Path

# 设置项目根目录（根据实际情况修改）
PROJECT_ROOT = Path("/content/Image-to-Image-Translation-Experiment")

# 如果项目不在 /content 下，请修改上面的路径
# 例如：PROJECT_ROOT = Path("/content/drive/MyDrive/Image-to-Image-Translation-Experiment")

if PROJECT_ROOT.exists():
    os.chdir(PROJECT_ROOT)
    print(f"✅ 项目目录：{PROJECT_ROOT}")
    
    # 如果是从 Git 克隆的，尝试更新代码（先清理 __pycache__）
    if (PROJECT_ROOT / ".git").exists():
        print("检测到 Git 仓库，清理 __pycache__ 后更新代码...")
        # 删除所有 __pycache__ 目录
        for pycache_dir in PROJECT_ROOT.rglob("__pycache__"):
            shutil.rmtree(pycache_dir)
            print(f"  删除：{pycache_dir}")
        
        # 现在可以安全地 pull
        try:
            result = subprocess.run(["git", "pull"], capture_output=True, text=True)
            if result.returncode == 0:
                print("✅ 代码更新成功")
            else:
                print(f"⚠️  Git pull 输出：{result.stdout}")
                print(f"⚠️  Git pull 错误：{result.stderr}")
        except Exception as e:
            print(f"⚠️  Git pull 失败：{e}，但可以继续使用当前代码")
else:
    print(f"⚠️  项目目录不存在：{PROJECT_ROOT}")
    print("请先上传代码或克隆仓库")


## 4. 解压数据文件（data.zip）

如果你已经将 `data.zip` 上传到 Google Drive，请先解压它。


In [ ]:
# ⚠️ 重要：请修改下面的路径为你的 data.zip 在 Google Drive 中的位置
# 例如：如果 data.zip 在 Drive 根目录，路径是：/content/drive/MyDrive/data.zip
DRIVE_DATA_ZIP = "/content/drive/MyDrive/data.zip"

import zipfile
from pathlib import Path

# 检查 zip 文件是否存在
if Path(DRIVE_DATA_ZIP).exists():
    print(f"✅ 找到 data.zip：{DRIVE_DATA_ZIP}")
    
    # 解压到项目目录下的 data 文件夹
    project_data_dir = PROJECT_ROOT / "data"
    project_data_dir.mkdir(exist_ok=True)
    
    print(f"正在解压到：{project_data_dir}")
    with zipfile.ZipFile(DRIVE_DATA_ZIP, 'r') as zip_ref:
        zip_ref.extractall(project_data_dir)
    
    print("✅ 解压完成")
    
    # 验证解压后的结构
    required_dirs = [
        "processed/train/photo",
        "processed/train/label",
        "processed/val/photo",
        "processed/val/label",
        "splits"
    ]
    
    missing_dirs = []
    for dir_name in required_dirs:
        if not (project_data_dir / dir_name).exists():
            missing_dirs.append(dir_name)
    
    if missing_dirs:
        print(f"⚠️  解压后缺少以下目录：{missing_dirs}")
        print("请检查 zip 文件结构是否正确")
    else:
        print("✅ 解压后的目录结构正确")
        
        # 检查划分文件
        split_file = project_data_dir / "splits" / "cityscapes_split_seed42.json"
        if split_file.exists():
            print(f"✅ 划分文件存在：{split_file}")
        else:
            print(f"⚠️  划分文件不存在：{split_file}")
            
        # 统计文件数量
        train_photos = list((project_data_dir / 'processed' / 'train' / 'photo').glob('*.jpg'))
        val_photos = list((project_data_dir / 'processed' / 'val' / 'photo').glob('*.jpg'))
        print(f"   训练集 photo 数量：{len(train_photos)} 张")
        print(f"   验证集 photo 数量：{len(val_photos)} 张")
else:
    print(f"⚠️  data.zip 不存在：{DRIVE_DATA_ZIP}")
    print("请检查路径是否正确，或先上传 data.zip 到 Drive")


## 5. 设置数据路径（如果数据已解压，可跳过此步骤）

如果你已经解压了 data.zip（上面的步骤），数据已经在项目目录下了，可以跳过这个步骤。

如果你使用的是其他方式准备的数据，请在这里设置路径。


## 6. 创建数据软链接（如果数据不在项目目录下）

如果数据在 Google Drive 的其他位置（而不是项目目录下），需要创建软链接。


In [ ]:
# ⚠️ 重要：请修改下面的路径为你的 Google Drive 数据目录
# 例如：/content/drive/MyDrive/Cityscapes/Image-to-Image-Translation-Experiment/data
DRIVE_DATA_PATH = "/content/drive/MyDrive/Cityscapes/Image-to-Image-Translation-Experiment/data"

import os
from pathlib import Path

# 检查 Drive 数据路径是否存在
drive_data_path = Path(DRIVE_DATA_PATH)
if not drive_data_path.exists():
    print(f"⚠️  Drive 数据路径不存在：{DRIVE_DATA_PATH}")
    print("请检查路径是否正确，或先上传数据到 Drive")
else:
    print(f"✅ Drive 数据路径存在：{DRIVE_DATA_PATH}")
    
    # 检查必要的子目录
    required_dirs = [
        "processed/train/photo",
        "processed/train/label",
        "processed/val/photo",
        "processed/val/label",
        "splits"
    ]
    
    missing_dirs = []
    for dir_name in required_dirs:
        if not (drive_data_path / dir_name).exists():
            missing_dirs.append(dir_name)
    
    if missing_dirs:
        print(f"⚠️  缺少以下目录：{missing_dirs}")
    else:
        print("✅ 所有必要的数据目录都存在")
    
    # 检查划分文件
    split_file = drive_data_path / "splits" / "cityscapes_split_seed42.json"
    if split_file.exists():
        print(f"✅ 划分文件存在：{split_file}")
    else:
        print(f"⚠️  划分文件不存在：{split_file}")


In [ ]:
# 在项目根目录下创建指向 Drive 数据目录的软链接
import os
from pathlib import Path

project_data_dir = PROJECT_ROOT / "data"

# 如果已存在 data 目录（可能是软链接或真实目录），先删除
if project_data_dir.exists() or project_data_dir.is_symlink():
    if project_data_dir.is_symlink():
        project_data_dir.unlink()
        print(f"删除旧的软链接：{project_data_dir}")
    else:
        print(f"⚠️  {project_data_dir} 已存在且不是软链接，请手动处理")

# 创建软链接
if not project_data_dir.exists():
    os.symlink(DRIVE_DATA_PATH, project_data_dir)
    print(f"✅ 创建软链接：{project_data_dir} -> {DRIVE_DATA_PATH}")
else:
    print(f"✅ 数据目录已存在：{project_data_dir}")

# 验证软链接
if (project_data_dir / "processed" / "train" / "photo").exists():
    print("✅ 数据软链接验证成功")
    train_photos = list((project_data_dir / 'processed' / 'train' / 'photo').glob('*.jpg'))
    val_photos = list((project_data_dir / 'processed' / 'val' / 'photo').glob('*.jpg'))
    print(f"   训练集 photo 数量：{len(train_photos)} 张")
    print(f"   验证集 photo 数量：{len(val_photos)} 张")
else:
    print("⚠️  数据软链接验证失败，请检查路径")


In [ ]:
import sys
from pathlib import Path

# 将项目根目录添加到 Python 路径
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print(f"✅ Python 路径已设置")
print(f"   项目根目录：{PROJECT_ROOT}")
print(f"   sys.path[0]：{sys.path[0]}")

# 验证导入
try:
    from src.models.generator import UNetGenerator
    from src.models.discriminator import PatchGANDiscriminator
    from src.losses.pix2pix_losses import pix2pix_generator_loss
    print("✅ 模块导入成功")
except ImportError as e:
    print(f"⚠️  模块导入失败：{e}")
    print("请检查项目结构是否正确")


## 7. 检查 GPU 可用性


In [ ]:
import torch

print(f"PyTorch 版本：{torch.__version__}")
print(f"CUDA 可用：{torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA 版本：{torch.version.cuda}")
    print(f"GPU 设备：{torch.cuda.get_device_name(0)}")
    print(f"GPU 数量：{torch.cuda.device_count()}")
    print("✅ GPU 可用，训练将使用 GPU")
else:
    print("⚠️  GPU 不可用，训练将使用 CPU（会很慢）")
    print("建议：在 Colab 中点击 Runtime -> Change runtime type -> Hardware accelerator -> GPU")


## 8. 运行 E2 实验：Pix2Pix (L1 + GAN)

### 8.1 快速测试（5 epochs，用于验证环境）


## ⚠️ 重要：免费版 Colab 限制说明

**免费版 Colab 的限制：**
- ⏱️ **运行时间限制**：每次会话最长约 **12 小时**，之后会自动断开
- 💤 **不活动断开**：如果约 **90 分钟**没有操作，会话可能被断开
- 🔄 **资源抢占**：如果 GPU 资源紧张，可能会被抢占

**应对策略：**
1. ✅ **自动保存 checkpoint**：训练脚本会每 10 个 epoch 自动保存 checkpoint
2. ✅ **保存到 Google Drive**：定期将结果复制到 Drive，避免丢失
3. ✅ **恢复训练功能**：如果训练中断，可以从 checkpoint 恢复继续训练
4. ✅ **分段训练**：可以将 100 epochs 分成多次运行（例如：每次 20-30 epochs）

**建议的训练流程：**
- 第一次运行：训练 20-30 epochs，保存 checkpoint
- 如果中断：使用 `--resume` 参数从最后一个 checkpoint 恢复
- 定期备份：每完成一段训练后，将结果复制到 Drive


In [ ]:
# 快速测试：运行 5 个 epoch 验证环境是否正确
%cd {PROJECT_ROOT}

!python src/training/train_pix2pix.py \
  --data-root data \
  --split-index data/splits/cityscapes_split_seed42.json \
  --epochs 5 \
  --batch-size 1 \
  --exp-name pix2pix_l1_gan_strong_debug \
  --aug-mode strong \
  --lambda-l1 100 \
  --num-val-samples 5

print("\n✅ 快速测试完成")


### 8.2 正式训练（200 epochs，完整 E2 实验）

**注意**：100 epochs 训练时间较长（根据 GPU 性能，可能需要 3-6 小时）。

**分段训练建议（适合免费版 Colab）：**
- 方案1：一次性训练 100 epochs（如果时间允许）
- 方案2：分段训练（推荐）
  - 第一次：训练 30 epochs
  - 第二次：从 checkpoint 恢复，继续训练 30 epochs
  - 第三次：从 checkpoint 恢复，继续训练 40 epochs
  - 每次训练后记得保存到 Drive

**如果训练中断：**
- 使用下面的"恢复训练"代码块，从最后一个 checkpoint 继续


In [ ]:
# 正式 E2 实验：100 epochs（完整训练）
# 如果训练中断，可以使用下面的"恢复训练"代码块继续

import os
os.chdir(PROJECT_ROOT)

!python src/training/train_pix2pix.py \
  --data-root data \
  --split-index data/splits/cityscapes_split_seed42.json \
  --epochs 100 \
  --batch-size 1 \
  --exp-name pix2pix_l1_gan_strong_e100 \
  --aug-mode strong \
  --lambda-l1 100 \
  --lr 2e-4 \
  --start-decay-epoch 50 \
  --num-val-samples 10 \
  --save-interval 10

print("\n✅ E2 实验训练完成（100 epochs）")


### 8.3 恢复训练（如果训练中断）

如果训练中断，可以使用这个代码块从最后一个 checkpoint 恢复训练。


In [ ]:
# 恢复训练：从最后一个 checkpoint 继续
# 这个代码会自动找到最新的 checkpoint 并恢复训练

import os
from pathlib import Path
import glob

os.chdir(PROJECT_ROOT)

exp_name = "pix2pix_l1_gan_strong_e100"
checkpoint_dir = PROJECT_ROOT / "outputs" / exp_name / "checkpoints"

# 查找最新的 checkpoint
checkpoint_files = list(checkpoint_dir.glob("checkpoint_epoch_*.pth"))
if checkpoint_files:
    # 按 epoch 编号排序，找到最新的
    checkpoint_files.sort(key=lambda x: int(x.stem.split("_")[-1]))
    latest_checkpoint = checkpoint_files[-1]
    
    print(f"📂 找到最新的 checkpoint: {latest_checkpoint}")
    print(f"   从该 checkpoint 恢复训练...")
    
    # 继续训练剩余的 epochs（假设总共要训练 100 epochs）
    # 脚本会自动从 checkpoint 中读取已训练的 epoch 数
    !python src/training/train_pix2pix.py \
      --data-root data \
      --split-index data/splits/cityscapes_split_seed42.json \
      --epochs 100 \
      --batch-size 1 \
      --exp-name {exp_name} \
      --aug-mode strong \
      --lambda-l1 100 \
      --lr 2e-4 \
      --start-decay-epoch 50 \
      --num-val-samples 10 \
      --save-interval 10 \
      --resume {latest_checkpoint}
    
    print("\n✅ 训练已恢复并继续")
else:
    print("⚠️  未找到 checkpoint，请先运行初始训练")


### 8.4 分段训练示例（推荐：每次 30 epochs）

如果担心训练时间太长，可以分段训练：


In [ ]:
# 分段训练：第一次运行（epochs 1-30）
# 完成后保存到 Drive，下次会话继续训练 31-60，最后训练 61-100

import os
os.chdir(PROJECT_ROOT)

# 第一次：训练前 30 epochs
!python src/training/train_pix2pix.py \
  --data-root data \
  --split-index data/splits/cityscapes_split_seed42.json \
  --epochs 30 \
  --batch-size 1 \
  --exp-name pix2pix_l1_gan_strong_e100 \
  --aug-mode strong \
  --lambda-l1 100 \
  --lr 2e-4 \
  --start-decay-epoch 50 \
  --num-val-samples 10 \
  --save-interval 10

print("\n✅ 第一阶段训练完成（epochs 1-30）")
print("💾 请运行下面的代码将结果保存到 Drive，然后下次会话继续训练")


In [ ]:
# 分段训练：第二次运行（从 epoch 30 继续到 60）
# 使用上面的"恢复训练"代码块，但修改 epochs 参数
# 或者直接运行：

import os
from pathlib import Path

os.chdir(PROJECT_ROOT)

exp_name = "pix2pix_l1_gan_strong_e100"
checkpoint_dir = PROJECT_ROOT / "outputs" / exp_name / "checkpoints"
latest_checkpoint = sorted(checkpoint_dir.glob("checkpoint_epoch_*.pth"), 
                           key=lambda x: int(x.stem.split("_")[-1]))[-1]

print(f"📂 从 checkpoint 恢复: {latest_checkpoint}")

!python src/training/train_pix2pix.py \
  --data-root data \
  --split-index data/splits/cityscapes_split_seed42.json \
  --epochs 60 \
  --batch-size 1 \
  --exp-name {exp_name} \
  --aug-mode strong \
  --lambda-l1 100 \
  --lr 2e-4 \
  --start-decay-epoch 50 \
  --num-val-samples 10 \
  --save-interval 10 \
  --resume {latest_checkpoint}

print("\n✅ 第二阶段训练完成（epochs 31-60）")


In [ ]:
# 分段训练：第三次运行（从 epoch 60 继续到 100，完成训练）
# 使用上面的"恢复训练"代码块，修改 epochs 为 100

import os
from pathlib import Path

os.chdir(PROJECT_ROOT)

exp_name = "pix2pix_l1_gan_strong_e100"
checkpoint_dir = PROJECT_ROOT / "outputs" / exp_name / "checkpoints"
latest_checkpoint = sorted(checkpoint_dir.glob("checkpoint_epoch_*.pth"), 
                           key=lambda x: int(x.stem.split("_")[-1]))[-1]

print(f"📂 从 checkpoint 恢复: {latest_checkpoint}")

!python src/training/train_pix2pix.py \
  --data-root data \
  --split-index data/splits/cityscapes_split_seed42.json \
  --epochs 100 \
  --batch-size 1 \
  --exp-name {exp_name} \
  --aug-mode strong \
  --lambda-l1 100 \
  --lr 2e-4 \
  --start-decay-epoch 50 \
  --num-val-samples 10 \
  --save-interval 10 \
  --resume {latest_checkpoint}

print("\n✅ 第三阶段训练完成（epochs 61-100）")
print("🎉 完整训练已完成！")


## 9. 保存结果到 Google Drive（重要：防止数据丢失）

**建议：**
- 每次训练完成后，立即运行下面的代码保存到 Drive
- 如果训练中断，checkpoint 已经保存在本地，可以恢复训练后再保存
- 保存到 Drive 后，下次会话可以从 Drive 恢复 checkpoint


In [ ]:
# 将训练结果（checkpoints、images、logs）复制到 Google Drive
# 这样即使 Colab 会话结束，结果也不会丢失

import shutil
from pathlib import Path

outputs_dir = PROJECT_ROOT / "outputs"
drive_outputs_dir = Path("/content/drive/MyDrive/Cityscapes/Image-to-Image-Translation-Experiment/outputs")

if outputs_dir.exists():
    # 创建 Drive 输出目录
    drive_outputs_dir.mkdir(parents=True, exist_ok=True)
    
    print(f"📦 正在复制结果到 Drive：{drive_outputs_dir}")
    
    # 逐个复制实验目录
    for exp_dir in outputs_dir.iterdir():
        if exp_dir.is_dir():
            dest_dir = drive_outputs_dir / exp_dir.name
            
            # 如果目标目录已存在，增量更新（只复制新文件）
            if dest_dir.exists():
                print(f"  🔄 更新已存在的目录：{exp_dir.name}")
                # 复制 checkpoints（增量）
                if (exp_dir / "checkpoints").exists():
                    (dest_dir / "checkpoints").mkdir(exist_ok=True, parents=True)
                    for ckpt in (exp_dir / "checkpoints").glob("*.pth"):
                        dest_ckpt = dest_dir / "checkpoints" / ckpt.name
                        if not dest_ckpt.exists() or ckpt.stat().st_mtime > dest_ckpt.stat().st_mtime:
                            shutil.copy2(ckpt, dest_ckpt)
                            print(f"    ✅ 已复制 checkpoint: {ckpt.name}")
                
                # 复制 images（增量）
                if (exp_dir / "images").exists():
                    (dest_dir / "images").mkdir(exist_ok=True, parents=True)
                    for img in (exp_dir / "images").glob("*.png"):
                        dest_img = dest_dir / "images" / img.name
                        if not dest_img.exists() or img.stat().st_mtime > dest_img.stat().st_mtime:
                            shutil.copy2(img, dest_img)
                
                # 复制 logs（覆盖）
                if (exp_dir / "logs").exists():
                    shutil.copytree(exp_dir / "logs", dest_dir / "logs", dirs_exist_ok=True)
            else:
                # 全新复制
                shutil.copytree(exp_dir, dest_dir)
                print(f"  ✅ 已复制新目录：{exp_dir.name}")
    
    print(f"\n✅ 结果已保存到 Drive：{drive_outputs_dir}")
    print(f"💡 下次会话可以从 Drive 恢复 checkpoint 继续训练")
else:
    print(f"⚠️  输出目录不存在：{outputs_dir}")
    print("请先运行训练脚本")


### 9.1 从 Google Drive 恢复 checkpoint（如果本地文件丢失）

如果 Colab 会话断开后重新开始，可以从 Drive 恢复 checkpoint：


In [ ]:
# 从 Google Drive 恢复 checkpoint 到本地
# 如果本地 outputs 目录被清空，可以从 Drive 恢复

import shutil
from pathlib import Path

drive_outputs_dir = Path("/content/drive/MyDrive/Cityscapes/Image-to-Image-Translation-Experiment/outputs")
local_outputs_dir = PROJECT_ROOT / "outputs"

exp_name = "pix2pix_l1_gan_strong_e100"
drive_exp_dir = drive_outputs_dir / exp_name
local_exp_dir = local_outputs_dir / exp_name

if drive_exp_dir.exists():
    print(f"📥 从 Drive 恢复：{drive_exp_dir}")
    
    # 创建本地目录
    local_exp_dir.mkdir(parents=True, exist_ok=True)
    
    # 恢复 checkpoints
    if (drive_exp_dir / "checkpoints").exists():
        (local_exp_dir / "checkpoints").mkdir(exist_ok=True, parents=True)
        for ckpt in (drive_exp_dir / "checkpoints").glob("*.pth"):
            shutil.copy2(ckpt, local_exp_dir / "checkpoints" / ckpt.name)
            print(f"  ✅ 已恢复 checkpoint: {ckpt.name}")
    
    # 恢复 logs
    if (drive_exp_dir / "logs").exists():
        shutil.copytree(drive_exp_dir / "logs", local_exp_dir / "logs", dirs_exist_ok=True)
        print(f"  ✅ 已恢复 logs")
    
    print(f"\n✅ 已从 Drive 恢复到本地：{local_exp_dir}")
    print(f"💡 现在可以使用上面的'恢复训练'代码块继续训练")
else:
    print(f"⚠️  Drive 中未找到实验目录：{drive_exp_dir}")
    print("请先运行训练并保存到 Drive")


In [ ]:
# 查看训练历史（loss 曲线等）
import json
from pathlib import Path
import matplotlib.pyplot as plt

history_file = PROJECT_ROOT / "outputs" / "pix2pix_l1_gan_strong" / "logs" / "history_pix2pix.json"

if history_file.exists():
    with open(history_file, 'r', encoding='utf-8') as f:
        history = json.load(f)
    
    print("训练历史摘要：")
    print(f"  总 epoch 数：{len(history.get('train_g_total', []))}")
    if history.get('val_psnr'):
        print(f"  最佳 Val PSNR：{max(history.get('val_psnr', [0])):.4f}")
    if history.get('val_ssim'):
        print(f"  最佳 Val SSIM：{max(history.get('val_ssim', [0])):.4f}")
    if history.get('val_l1'):
        print(f"  最低 Val L1：{min(history.get('val_l1', [float('inf')])):.4f}")
    
    # 绘制损失曲线（简单示例）
    epochs = list(range(1, len(history.get('train_g_total', [])) + 1))
    
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    if history.get('train_g_total'):
        plt.plot(epochs, history.get('train_g_total', []), label='G Total')
    if history.get('train_g_l1'):
        plt.plot(epochs, history.get('train_g_l1', []), label='G L1')
    if history.get('train_g_gan'):
        plt.plot(epochs, history.get('train_g_gan', []), label='G GAN')
    if history.get('train_d_loss'):
        plt.plot(epochs, history.get('train_d_loss', []), label='D Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Losses')
    plt.legend()
    plt.grid(True)
    
    plt.subplot(1, 2, 2)
    if history.get('val_psnr'):
        plt.plot(epochs, history.get('val_psnr', []), label='PSNR')
    if history.get('val_ssim'):
        plt.plot(epochs, history.get('val_ssim', []), label='SSIM')
    plt.xlabel('Epoch')
    plt.ylabel('Metric')
    plt.title('Validation Metrics')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()
else:
    print(f"⚠️  训练历史文件不存在：{history_file}")
